In [1]:
%matplotlib inline
%load_ext autoreload
%load_ext memory_profiler
%autoreload 2

import sys, os
# sys.path.append('../largescale_smc/')
# sys.path.append('../third_party')

In [88]:
import numpy as np   
import pandas as pd
import pdb
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from tqdm import tqdm
import sys

from time import time
from largescale_smc.largescale_missingness import *
from largescale_smc.largescale_utils import *
from largescale_smc.largescale_method import *

In [89]:
from surprise import SVD
from surprise import accuracy

In [90]:
%%memit
start_time = time()
#file_path = os.path.expanduser("../data/ml-100k/u.data")
file_path = os.path.expanduser("../data/ml-10m/ratings_no_timestamp.dat")
reader = DataReader(line_format="user item rating", sep="::")
splitter = DataSplitter(file_path, reader)
train_samples, calib_samples, missing_idxs = splitter.sample_train_calib(k=5, random_state=0)
print(f"Elapsed time: {time()-start_time} seconds")

10000054it [00:16, 619760.72it/s]

Loaded 10000054 ratings from 69878 users and 10677 items.


Elapsed time: 40.97227096557617 seconds
peak memory: 6169.39 MiB, increment: 2530.52 MiB


In [5]:
%%memit
random_state=0
start_time = time()
trainset = construct_trainset(train_samples, reader.rating_scale)
algo = SVD(n_factors=10, n_epochs=10, biased=False, verbose=True)
algo.fit(trainset)
print(f"Fitting time: {time()-start_time} seconds")

# start_time = time()
# # Then compute RMSE
# predictions = algo.test(raw_testset)
# accuracy.rmse(predictions)
# print(f"Testing time: {time()-start_time} seconds")

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Fitting time: 67.09283328056335 seconds
peak memory: 4031.85 MiB, increment: 1555.52 MiB


In [6]:
%%memit
start_time = time()
w_obs=LogisticMFProbs(random_state=random_state)
w_obs.fit(train_samples+calib_samples)
print(f"Elapsed time: {time()-start_time} seconds")

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:21<00:00,  1.40it/s]


Elapsed time: 56.185293197631836 seconds
peak memory: 4630.21 MiB, increment: 742.57 MiB


In [107]:
%%memit
start_time = time()
k=5
Mhat = create_batch_rating_predictor(algo)
ci_method = SimulCI_ls(splitter.config, calib_samples, k, Mhat, w_obs, num_workers=6)
print(f"Elapsed time: {time()-start_time} seconds")

Elapsed time: 0.07501602172851562 seconds
peak memory: 5496.71 MiB, increment: 0.18 MiB


In [108]:
calib_idxs = [(u, i) for u,i,r in calib_samples]
calib_ratings = [r for u,i,r in calib_samples]

In [110]:
%%memit
start_time = time()
alpha = 0.1
allow_inf= False
df = ci_method.get_CI(calib_idxs, alpha, allow_inf=allow_inf)
print(f"Elapsed time: {time()-start_time} seconds")

Computing conformal prediction intervals for 2000 test queries...
Start computing universal quantities invariant to the test query...
Number of worker:6.


Universal quantities: 100%|██████████████████████████████████████████████████████| 10677/10677 [03:02<00:00, 58.65it/s]


Done!
Elapsed time: 251.93575382232666 seconds


CI: 100%|██████████████████████████████████████████████████████████████████████████| 2000/2000 [03:10<00:00, 10.52it/s]

Done!
Elapsed time: 442.12897753715515 seconds
peak memory: 6264.12 MiB, increment: 768.01 MiB


In [111]:
lower, upper, is_inf, is_impossible = df.loc[0].lower, df.loc[0].upper, df.loc[0].is_inf, df.loc[0].is_impossible

In [112]:
def clip_intervals(lower, upper):
    lower[lower <= 1] = 1
    upper[upper >= 5] = 5
    return lower, upper

In [113]:
lower, upper = clip_intervals(lower, upper)

In [114]:
res = evaluate_SCI_ls(lower, upper, k, calib_ratings, is_inf, is_impossible)

Filtered out 0 impossible queries out of 2000 queries.
Evaluating 2000 queries...
Done!


In [115]:
res

,Query_coverage,Coverage,Size,metric,Impossible_prop,Inf_prop
0,0.91,0.9798,3.596397,mean,0.0,0.0


In [122]:
test_idxs =  splitter.sample_test(k, max_n_test=200, random_state=0)

In [123]:
df = ci_method.get_CI(test_idxs, alpha, allow_inf=allow_inf)

Computing conformal prediction intervals for 200 test queries...
Skipping some duplicated computations: w_test is unchanged.


CI: 100%|████████████████████████████████████████████████████████████████████████████| 200/200 [00:22<00:00,  9.02it/s]

Done!


In [124]:
lower, upper, is_inf, is_impossible = df.loc[0].lower, df.loc[0].upper, df.loc[0].is_inf, df.loc[0].is_impossible
lower, upper = clip_intervals(lower, upper)

In [125]:
res = evaluate_SCI_ls(lower, upper, k, np.zeros(len(test_idxs)), is_inf, is_impossible)

Filtered out 0 impossible queries out of 200 queries.
Evaluating 200 queries...
Done!


In [126]:
res

,Query_coverage,Coverage,Size,metric,Impossible_prop,Inf_prop
0,0.0,0.0,3.651051,mean,0.0,0.04


In [129]:
np.arange(2,6)

array([2, 3, 4, 5])